# Assignments: Prepping for Modeling

## 1. Create a Single Table

Hi there,

We just finished collecting our Q2 book sales data. Can you help us create one giant table that includes:

* April, May and June’s book sales
* Customer data

Each table has a field called customer_id that links all the tables together.

I’ve attached four files.

Thanks!\
Brooke

In [1]:
# Read all four files into a Jupyter Notebook
import pandas as pd

april = pd.read_excel('../Data/Book_Sales_April.xlsx')
may = pd.read_excel('../Data/Book_Sales_May.xlsx')
june = pd.read_excel('../Data/Book_Sales_June.xlsx')
customers = pd.read_csv('../Data/Book_Customers.csv')

In [2]:
# Concatenate the April, May and June book sales DataFrames
sales = pd.concat([april, may, june]).reset_index(drop=True)
sales.head()

,Purchase ID,Customer ID,Book,Audience,Genre,Price,Book Rating,Purchase Location,Purchase Date
0,401,101,Unicorns Are Cool,Children,Fiction,4.99,3.6,Online,2023-04-01
1,402,101,I Love Unicorns,Children,Fiction,5.99,4.1,Online,2023-04-01
2,403,102,All About Turtles,Teens,Non-Fiction,19.99,3.3,In Person,2023-04-02
3,404,102,All About Whales,Teens,Non-Fiction,19.99,2.5,In Person,2023-04-02
4,405,102,All About Dolphins,Teens,Non-Fiction,19.99,4.2,In Person,2023-04-02


In [3]:
# Join the newly created book sales DataFrame with the customers DataFrame on customer_id
# Which type of join do you think would work best here?
customers.head()

,Customer ID,Customer Name,Age,Has School Aged Children,Has_Pets
0,101,Alexander,36,Yes,No
1,102,Mason,16,No,No
2,103,Ethan,55,No,No
3,104,Daniel,70,No,No
4,105,Michael,18,No,Yes


In [4]:
df = sales.merge(customers, how='left', on='Customer ID')
df.head()

,Purchase ID,Customer ID,Book,Audience,Genre,Price,Book Rating,Purchase Location,Purchase Date,Customer Name,Age,Has School Aged Children,Has_Pets
0,401,101,Unicorns Are Cool,Children,Fiction,4.99,3.6,Online,2023-04-01,Alexander,36,Yes,No
1,402,101,I Love Unicorns,Children,Fiction,5.99,4.1,Online,2023-04-01,Alexander,36,Yes,No
2,403,102,All About Turtles,Teens,Non-Fiction,19.99,3.3,In Person,2023-04-02,Mason,16,No,No
3,404,102,All About Whales,Teens,Non-Fiction,19.99,2.5,In Person,2023-04-02,Mason,16,No,No
4,405,102,All About Dolphins,Teens,Non-Fiction,19.99,4.2,In Person,2023-04-02,Mason,16,No,No


## 2. Prepare Rows for Modeling

Hi again,

We are trying to predict which customers will purchase a book this month.

Can you reformat the data you compiled earlier this week so that it’s ready to be input into a model, with each row representing a customer instead of a purchase?

Thanks!\
Brooke

**problem solving**
>What should one row of data look like for the model?
- attributes - customer 
- purchased in June or not - y
- summary stats about April and May - granularity - x

In [5]:
# Create a column called 'June Purchases' that sums up all purchases in June
#june.head()
#aggregate the june data using any columns for Customer Id rows
june_purchases = (june.groupby('Customer ID')['Book']
                     .count()
                     .rename('June Purchases')
                     .to_frame()            #to have the series in df
                     .reset_index())
june_purchases.head()

,Customer ID,June Purchases
0,101,2
1,103,1
2,106,1
3,108,1
4,111,2


In [6]:
# Create a column called 'Total Spend' that sums up the prices of all the books that were purchased in April and May
df_april_may = df[df['Purchase Date'].dt.month < 6]
df_april_may.tail()

,Purchase ID,Customer ID,Book,Audience,Genre,Price,Book Rating,Purchase Location,Purchase Date,Customer Name,Age,Has School Aged Children,Has_Pets
26,427,106,Which Dino is Best?,Children,Non-Fiction,8.99,2.6,In Person,2023-05-22,Matthew,40,Yes,No
27,428,101,Unicorns Are The Best,Children,Fiction,4.99,2.3,Online,2023-05-23,Alexander,36,Yes,No
28,429,111,Star Battles: Part 3,Teens,Fiction,13.50,4.4,Online,2023-05-24,Carter,17,No,No
29,430,112,Our Wonderous Planet,Teens,Non-Fiction,17.75,4.5,In Person,2023-05-25,Owen,17,No,No
30,431,116,Who's There,Adults,Fiction,24.99,2.1,Online,2023-05-26,Emily,51,No,No


In [7]:
total_spend = (df_april_may.groupby('Customer ID')['Price']
                .sum()
                .rename('Total Spend')
                .to_frame()
                .reset_index())
total_spend.head()

,Customer ID,Total Spend
0,101,33.71
1,102,79.96
2,103,55.20
3,104,42.32
4,105,21.50


In [8]:
# Combine the 'June Purchases' and 'Total Spend' columns into a single DataFrame for modeling
model_df = total_spend.merge(june_purchases, how='left').fillna(0)
model_df.head()

,Customer ID,Total Spend,June Purchases
0,101,33.71,2.0
1,102,79.96,0.0
2,103,55.20,1.0
3,104,42.32,0.0
4,105,21.50,0.0


## 3. Prepare Columns for Modeling

Hi again,

Thanks for your help earlier this week! We just learned that we also have to make all of the data numeric before inputting it into a predictive model.

Can you turn the Audience text field into a numeric field?

Thanks!\
Brooke

In [9]:
df.head(2)

,Purchase ID,Customer ID,Book,Audience,Genre,Price,Book Rating,Purchase Location,Purchase Date,Customer Name,Age,Has School Aged Children,Has_Pets
0,401,101,Unicorns Are Cool,Children,Fiction,4.99,3.6,Online,2023-04-01,Alexander,36,Yes,No
1,402,101,I Love Unicorns,Children,Fiction,5.99,4.1,Online,2023-04-01,Alexander,36,Yes,No


In [10]:
df_april_may.Audience.value_counts()

Audience
Adults      12
Children    10
Teens        9
Name: count, dtype: int64

In [11]:
# Change the Audience field into a dummy variable
audience_dummies = pd.get_dummies(df_april_may.Audience)
audience_dummies.head()

,Adults,Children,Teens
0,False,True,False
1,False,True,False
2,False,False,True
3,False,False,True
4,False,False,True


In [12]:
#view the df_april_may to be joined
df_april_may['Customer ID'].head()

0    101
1    101
2    102
3    102
4    102
Name: Customer ID, dtype: int64

In [13]:
#join both dataframe - pd.concat as both indexes are identical
cust_audience = pd.concat([df_april_may['Customer ID'], audience_dummies], axis=1)
cust_audience.head() 

,Customer ID,Adults,Children,Teens
0,101,False,True,False
1,101,False,True,False
2,102,False,False,True
3,102,False,False,True
4,102,False,False,True


In [14]:
# Using the Audience dummy variables, create three new columns that contain
# the number of Adult / Children / Teen books purchased by each customer
book_types = cust_audience.groupby('Customer ID').sum().reset_index()
book_types.head()

,Customer ID,Adults,Children,Teens
0,101,0,5,0
1,102,0,0,4
2,103,2,0,0
3,104,2,0,0
4,105,1,0,0


In [15]:
# Combine the three new columns back with the customer-level data
model_df = model_df.merge(book_types, how='left')
model_df.head()

,Customer ID,Total Spend,June Purchases,Adults,Children,Teens
0,101,33.71,2.0,0,5,0
1,102,79.96,0.0,0,0,4
2,103,55.20,1.0,2,0,0
3,104,42.32,0.0,2,0,0
4,105,21.50,0.0,1,0,0


## 4. Feature Engineering

Hi again,

I have one final request for you.

As a reminder, our goal is to try and predict which customer will purchase a book this month.

Can you create new features that you think will do a good job making a prediction?

Thanks!\
Brooke

In [16]:
# what features could do a good job predicting june purchases?
## whether a customer has school-aged children
## percent of the time that a customer purchases online vs in person

In [17]:
customers.head()

,Customer ID,Customer Name,Age,Has School Aged Children,Has_Pets
0,101,Alexander,36,Yes,No
1,102,Mason,16,No,No
2,103,Ethan,55,No,No
3,104,Daniel,70,No,No
4,105,Michael,18,No,Yes


In [18]:
## whether a customer has school-aged children
has_kids = (pd.get_dummies(customers[['Customer ID', 'Has School Aged Children']], drop_first = True)
              .rename(columns = {'Has School Aged Children_Yes': 'Has Kids'}))
has_kids.head()

,Customer ID,Has Kids
0,101,True
1,102,False
2,103,False
3,104,False
4,105,False


In [19]:
# add it to the model dataframe
model_df = model_df.merge(has_kids, how='left', on='Customer ID')
model_df.head()

,Customer ID,Total Spend,June Purchases,Adults,Children,Teens,Has Kids
0,101,33.71,2.0,0,5,0,True
1,102,79.96,0.0,0,0,4,False
2,103,55.20,1.0,2,0,0,False
3,104,42.32,0.0,2,0,0,False
4,105,21.50,0.0,1,0,0,False


In [20]:
# make the purchase location numeric
location = pd.get_dummies(df_april_may['Purchase Location'])
location.head()

,In Person,Online
0,False,True
1,False,True
2,True,False
3,True,False
4,True,False


In [21]:
# add it on the original dataframe
location_df = pd.concat([df_april_may, location], axis=1)[['Customer ID', 'In Person', 'Online']]
location_df.head()

,Customer ID,In Person,Online
0,101,False,True
1,101,False,True
2,102,True,False
3,102,True,False
4,102,True,False


In [22]:
#group the locations by dataframe
location_group = location_df.groupby('Customer ID').sum().reset_index()
location_group.head()

,Customer ID,In Person,Online
0,101,0,5
1,102,3,1
2,103,2,0
3,104,1,1
4,105,0,1


In [23]:
# calculate the percent in person purchases by customer
in_person_pct = ((location_group['In Person'] / (location_group['In Person'] + location_group['Online'])*100)
                 .rename('Percent In Person'))
in_person_pct.head()

0      0.0
1     75.0
2    100.0
3     50.0
4      0.0
Name: Percent In Person, dtype: float64

In [24]:
#add it on the model dataframe
model_df = pd.concat([model_df, in_person_pct], axis=1)
model_df.head()

,Customer ID,Total Spend,June Purchases,Adults,Children,Teens,Has Kids,Percent In Person
0,101,33.71,2.0,0,5,0,True,0.0
1,102,79.96,0.0,0,0,4,False,75.0
2,103,55.20,1.0,2,0,0,False,100.0
3,104,42.32,0.0,2,0,0,False,50.0
4,105,21.50,0.0,1,0,0,False,0.0


## The Final DataFrame for Modeling

In [25]:
#our non-null, numeric dataframe with x and y variables
model_df

,Customer ID,Total Spend,June Purchases,Adults,Children,Teens,Has Kids,Percent In Person
0,101,33.71,2.0,0,5,0,True,0.000000
1,102,79.96,0.0,0,0,4,False,75.000000
2,103,55.20,1.0,2,0,0,False,100.000000
3,104,42.32,0.0,2,0,0,False,50.000000
4,105,21.50,0.0,1,0,0,False,0.000000
5,106,38.96,1.0,0,4,0,True,75.000000
6,107,25.99,0.0,1,0,0,True,0.000000
7,108,9.20,1.0,0,1,0,True,100.000000
8,109,22.72,0.0,1,0,0,False,0.000000
9,110,21.58,0.0,1,0,0,False,100.000000


In [26]:
# y = June purchases
# x = Total spend through Percent in Person
# would need to remove Customer ID for modeling

In [27]:
# the final dataframe that is ready to be input into a model!
final_df = model_df[['June Purchases', 'Total Spend', 'Adults', 
                     'Children', 'Teens', 'Has Kids', 'Percent In Person']]
final_df

,June Purchases,Total Spend,Adults,Children,Teens,Has Kids,Percent In Person
0,2.0,33.71,0,5,0,True,0.000000
1,0.0,79.96,0,0,4,False,75.000000
2,1.0,55.20,2,0,0,False,100.000000
3,0.0,42.32,2,0,0,False,50.000000
4,0.0,21.50,1,0,0,False,0.000000
5,1.0,38.96,0,4,0,True,75.000000
6,0.0,25.99,1,0,0,True,0.000000
7,1.0,9.20,0,1,0,True,100.000000
8,0.0,22.72,1,0,0,False,0.000000
9,0.0,21.58,1,0,0,False,100.000000
